In [6]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [8]:
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
from queue import Queue

messages = Queue()
recordings = Queue()

record_button = widgets.Button(
    description="Record",
    disabled=False,
    button_style = "success",
    icon = "microphone"
)

stop_button = widgets.Button(
    description="Stop",
    disabled=False,
    button_style="warning",
    icon="stop"
)

output = widgets.Output()

def start_recording(data):
    messages.put(True)
    
    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()
        
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()
        
def stop_recording(data):
    with output:
        messages.get()
        display("Stopped...")
        
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button,stop_button,output)


Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()

In [11]:
%pip install pyaudio

     ---------------------------------------- 0.0/164.1 kB ? eta -:--:--
     ---------------------------------------- 0.0/164.1 kB ? eta -:--:--
     ---------------------------------------- 0.0/164.1 kB ? eta -:--:--
     -- ------------------------------------- 10.2/164.1 kB ? eta -:--:--
     -- ------------------------------------- 10.2/164.1 kB ? eta -:--:--
     -- ------------------------------------- 10.2/164.1 kB ? eta -:--:--
     ------------- ----------------------- 61.4/164.1 kB 297.7 kB/s eta 0:00:01
     ------------- ----------------------- 61.4/164.1 kB 297.7 kB/s eta 0:00:01
     ------------------ ------------------ 81.9/164.1 kB 269.5 kB/s eta 0:00:01
     ------------------ ------------------ 81.9/164.1 kB 269.5 kB/s eta 0:00:01
     ------------------ ------------------ 81.9/164.1 kB 269.5 kB/s eta 0:00:01
     ------------------ ------------------ 81.9/164.1 kB 269.5 kB/s eta 0:00:01
     ------------------ ------------------ 81.9/164.1 kB 269.5 kB/s eta 0:00:01

In [12]:
import pyaudio

p = pyaudio.PyAudio()
for i in range (p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Microphone Array (Realtek(R) Au', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Headset (TWS 5517)', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputChannels': 0, 'max

In [15]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()
    
    stream = p.open(format = AUDIO_FORMAT,
                    channels = CHANNELS,
                    rate= FRAME_RATE,
                    input = True,
                    input_device_index=1,
                    frames_per_buffer=chunk)
    frames= []
    
    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = []
            
    stream.stop_stream()
    stream.close()
    p.terminate()
            
    
    

In [8]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import subprocess
import json
from vosk import Model, KaldiRecognizer

model = Model(model_name="vosk-model-en-us-0.22")
rec= KaldiRecognizer(model,FRAME_RATE)
rec.SetWords(True)

def speech_recogition(output):
    while not messages.empty():
        frames = recordings.get()
        
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text=json.loads(result)["text"]
        